# Analyzing CIA Factbook Data Using SQL

<img src="https://www.northamptonapl.org/sites/default/files/cia_0.png">


## Introduction

In this project, I'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:

* population — the global population.
* population_growth — the annual population growth rate, as a percentage.
* area — the total land and water area.

In this guided project, I'll use SQL in Jupyter Notebook to analyze data from this database.

----

## Connect to DB File

In [49]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

----

## Return DB Info

In [50]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


----

## Data Overview

In [51]:
%%sql
SELECT *
  FROM facts
LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

* name — the name of the country.
* area— the country's total area (both land and water).
* area_land — the country's land area in square kilometers.
* area_water — the country's water area in square kilometers.
* population — the country's population.
* population_growth— the country's population growth as a percentage.
* birth_rate — the country's birth rate, or the number of births per year per 1,000 people.
* death_rate — the country's death rate, or the number of death per year per 1,000 people.

----

## Summary Statistics (or Finding Outliers)

In [52]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts;


Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


In [53]:
%%sql
SELECT name, MIN(population)
  FROM facts;

Done.


name,MIN(population)
Antarctica,0


Here is an explanation for the results of countries with a min population of 0 (Antarctica).

<img src='https://s3.amazonaws.com/dq-content/257/fb_antarctica.png'>

In [54]:
%%sql
SELECT name, MAX(population)
  FROM facts;

Done.


name,MAX(population)
World,7256490011


Since our dataset should only consider countries, we will remove the 'World' entry as it skews our average.

----

## Remove Outlier 'World' - Not Relevant

In [55]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts
    WHERE name != 'World';

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


----

## Re-calculate Averages ("Hey, not you...'World'")

In [56]:
%%sql
SELECT AVG(population), AVG(area)
  FROM facts
    WHERE name != 'World';

Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


----

## Densely Populated Countries

These countries are densely populated based on above-average population and below average area.

In [57]:
%%sql
SELECT name as 'NAME', area as 'AREA', population as 'POPULATION'
    FROM facts
    WHERE population > (SELECT AVG(population)
                       FROM facts
                       WHERE name <> 'World') AND 
          area < (SELECT AVG(area)
                       FROM facts
                       WHERE name <> 'World');

Done.


NAME,AREA,POPULATION
Bangladesh,148460,168957745
Germany,357022,80854408
Iraq,438317,37056169
Italy,301340,61855120
Japan,377915,126919659
"Korea, South",99720,49115196
Morocco,446550,33322699
Philippines,300000,100998376
Poland,312685,38562189
Spain,505370,48146134


----

## Country with the Highest Population

In [58]:
%%sql
SELECT name, population
  FROM facts
    WHERE name != 'World'
        ORDER BY population DESC
        LIMIT 1;


Done.


name,population
China,1367485388


----

## Country with the Highest Growth Rate

In [59]:
%%sql
SELECT name, population_growth
  FROM facts
    WHERE name != 'World'
        ORDER BY population_growth DESC
        LIMIT 1;

Done.


name,population_growth
South Sudan,4.02


----

## Countries with the Highest Birth Rate

These countries will add the most people to their populations next year.


In [60]:
%%sql
SELECT name, birth_rate
  FROM facts
    WHERE name != 'World'
        ORDER BY birth_rate DESC
        LIMIT 10;

Done.


name,birth_rate
Niger,45.45
Mali,44.99
Uganda,43.79
Zambia,42.13
Burkina Faso,42.03
Burundi,42.01
Malawi,41.56
Somalia,40.45
Angola,38.78
Mozambique,38.58


----

## Countries with Higher Death Rate Than Birth Rate

These countries will lose the most people next year.


In [61]:
%%sql
SELECT name, death_rate, birth_rate
  FROM facts
    WHERE death_rate > birth_rate
        ORDER BY death_rate DESC;

Done.


name,death_rate,birth_rate
Ukraine,14.46,10.72
Bulgaria,14.44,8.92
Latvia,14.31,10.0
Lithuania,14.27,10.1
Russia,13.69,11.6
Serbia,13.66,9.08
Belarus,13.36,10.7
Hungary,12.73,9.16
Moldova,12.59,12.0
Estonia,12.4,10.51
